In [ ]:
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
import scipy.fftpack
%matplotlib inline

In [ ]:
from analysis.lib.nv import detection
folder= tb.latest_data('_EntangleXsweepY')
# folder = "D:/measuring/data/20170308/105316_PQ_single_click_ent_111no2_SIL2_phase_stab"
a = m2.M2Analysis(folder)

# stitch the different part and files together, make and save a plot of the data.

pid_counts_1 = a.g['adwindata']['pid_counts_1'].value
pid_counts_2 = a.g['adwindata']['pid_counts_2'].value
max_repetitions = a.g['adwindata']['store_index_stab'].value
g_0 = a.g.attrs['Phase_Msmt_g_0']
visibility = a.g.attrs['Phase_Msmt_Vis']

delay = a.g.attrs['count_int_time_stab']/1000.0
pid_cycles = a.g.attrs['pid_points']

# stitch
cosvals = [2*(float(n0)/(float(n0)+float(n1)*g_0)-0.5)*visibility for n0,n1 in zip(pid_counts_1,pid_counts_2)]
cosvals = [cosval if np.abs(cosval) < 1 else (1.0 * np.sign(cosval)) for cosval in cosvals]
angle1 = np.arccos(cosvals)

v_1 = (180.0*angle1/(np.pi))


print np.shape(v_1)
print pid_cycles
t = np.arange(0, (len(pid_counts_1)*delay), (delay))

# Angle
fig = plt.figure(figsize=(17,6))
ax = plt.subplot(111)
plt.plot(t, v_1, 'r')
plt.title('Phase of ZPL photons {0}'.format(folder))
plt.ylim([0,180])
ax.set_xlabel('elapsed time (milliseconds)')
ax.set_ylabel('angle ($^o$)')

fig.savefig(os.path.join(folder, 'trace_angle.png'))

print 'Phase standard deviation ', np.sqrt(np.var(v_1))

fig = plt.figure()
ax = plt.subplot(111)
hist, bins = np.histogram(v_1,bins= 100,normed = True)
width = np.diff(bins)
center = (bins[:-1] + bins[1:]) / 2
ax.bar(center, hist, align='center', width=width)
g_a = 0.0
g_x0 = 90
g_sigma = 15
g_A = 1/(np.sqrt(2 * np.pi) * g_sigma)

p0, fitfunc,fitfunc_str = common.fit_gauss(g_a, g_A, g_x0, g_sigma)
fit_result = fit.fit1d(center,hist, None, p0=p0, fitfunc=fitfunc,
                     ret=True,fixed=[])
plot.plot_fit1d(fit_result, np.linspace(center[0],center[-1],201), ax=ax, 
                    plot_data=False,print_info = True)
print 'x0, sigma ', fit_result['params_dict']['x0'] , fit_result['params_dict']['sigma'] 
# std dev
fig = plt.figure()
ax = plt.subplot(111)
plt.plot(np.sqrt(np.var(np.reshape(v_1,[pid_cycles,-1]),axis=1)))
plt.title('Standard deviation of Phase {0}'.format(folder))
ax.set_xlabel('pid cycles (-)')
ax.set_ylabel('std dev ($^o$)')

In [ ]:
# counts
fig = plt.figure(figsize=(17,6))
ax = plt.subplot(211)
# plt.plot(t, s, 'y')
plt.plot(t, pid_counts_1, 'r')
plt.title('Counts ZPL detector 1 {0}'.format(folder))
ax.set_xlabel('elapsed time (milliseconds)')
ax.set_ylabel('counts')
ax2 = plt.subplot(212)
plt.plot(t, pid_counts_2, 'r')
plt.title('Counts ZPL detector 2 {0}'.format(folder))
ax2.set_xlabel('elapsed time (milliseconds)')
ax2.set_ylabel('counts')
# ax3 = plt.subplot(313)
# plt.plot(t, w, 'r')
# plt.title('error signal {0}'.format(folder))
# ax3.set_xlabel('elapsed time (milliseconds)')
# ax3.set_ylabel('error signal')
plt.tight_layout()
# plt.show()

fig.savefig(os.path.join(folder, 'trace_counts.png'))zoom_1 = 5*(pid_cycles)
zoom_2 = 6*(pid_cycles)

# Zoomed counts
fig = plt.figure(figsize=(17,6))
ax = plt.subplot(211)
# plt.plot(t[0:zoom], s[0:zoom], 'y')
plt.plot(t[zoom_1:zoom_2], pid_counts_1[zoom_1:zoom_2], 'r')

plt.title('Counts ZPL detector 1'.format(folder))
ax.set_xlabel('elapsed time (milliseconds)')
ax.set_ylabel('counts')

ax2 = plt.subplot(212)
plt.plot(t[zoom_1:zoom_2], pid_counts_2[zoom_1:zoom_2], 'r')

plt.title('Counts ZPL detector 2'.format(folder))
ax2.set_xlabel('elapsed time (milliseconds)')
ax2.set_ylabel('counts')
plt.tight_layout()

fig.savefig(os.path.join(folder, 'trace_counts_zoomed.png'))

In [ ]:
# fourier transform of the sample part

pid_counts_1 = a.g['adwindata']['pid_counts_1'].value
sample_counts_1 = a.g['adwindata']['sampling_counts_1'].value
max_repetitions = a.g['adwindata']['completed_reps'].value

delay = a.g.attrs['count_int_time_stab']/1000.0
pid_cycles = a.g.attrs['pid_points']
sample_cycles = a.g.attrs['sample_points']

# stitch
v_1 = []
for i in xrange(max_repetitions):
    for j in xrange(pid_cycles):
         v_1.append(pid_counts_1[j+i*pid_cycles])
    for k in xrange(sample_cycles):
         v_1.append(sample_counts_1[k+i*sample_cycles])
yf = scipy.fftpack.fft(v_1)

# sample spacing
xf = np.linspace(0, 1.0/(2.0*delay*10**-4), (sample_cycles+pid_cycles)*max_repetitions/2)

index_max = [i for i,v in enumerate(yf) if v > 20000]
frq_max = [x * (1.0/(delay*10**-4)) / ((sample_cycles+pid_cycles)*max_repetitions) for x in index_max]
print 'peak frequencies are:'
print frq_max

norm1 = yf / np.linalg.norm(yf)

fig, ax = plt.subplots()
plt.ylim([0,20000])
ax.plot(xf, yf[:len(yf)/2])
plt.title('FFT {0}'.format(folder))
ax.set_xlabel('frequency (Hz)')
ax.set_ylabel('Amplitude (a.u.)')
# ax.set_yticklabels([])
fig.savefig(os.path.join(folder, 'fft.png'))